In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
# Clear Database and create connection
try:

    os.remove('../HomelessData.db')
    print("Cleared Database")
except Exception as E : 
    print("Could not clear database")
    print(E)
conn = sqlite3.connect('../HomelessData.db')
c = conn.cursor()

Could not clear database
[WinError 2] The system cannot find the file specified: '../HomelessData.db'


In [3]:
# Create ShelterAvalibility dataframe and database
for sheet in ["2020","2019","2018","2017","2016","2015","2014","2013"]: 
    try: 
        hic_df = pd.read_excel("../resources/2007-2020-HIC-Counts-by-CoC.xlsx",sheet,skiprows=[0],skipfooter=1)

        # Adjust Column
        hic_df.rename(columns = {'CoC Number' : 'Coc','Total Year-Round Beds (ES, TH, SH)' : 'Total','Total Year-Round Beds (ES)' : 'Emergency','Total Year-Round Beds (TH)' : 'Transitional','Total Year-Round Beds (SH)' : 'SafeHaven','Total Year-Round Beds (RRH)' : 'RapidRehousing','Total Year-Round Beds (PSH)' : 'PermanentSupportive','Total Year-Round Beds (OPH)' : 'PermanentOther'},inplace=True)
        hic_df = hic_df[["Coc", "Total", "Emergency", "Transitional", "SafeHaven", "RapidRehousing", "PermanentSupportive", "PermanentOther"]]
        hic_df["Year"] = sheet
        hic_df["CocYear"] = hic_df["Coc"] +" "+ hic_df["Year"]

        # Insert Into DB
        hic_df.to_sql("shelterAvalibility",con=conn,if_exists = 'append',index=False)
        print(f"Gathered Data from {sheet}")
    except Exception as E:
        print(f"Couldnt get data from {sheet}")
print("Finished")

Gathered Data from 2020
Gathered Data from 2019
Gathered Data from 2018
Gathered Data from 2017
Gathered Data from 2016
Gathered Data from 2015
Gathered Data from 2014
Couldnt get data from 2013
Finished


C:\Users\smshe\anaconda3\envs\mlenv\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [4]:
# Create PIT and coc dataframe and database
for sheet in ["2020","2019","2018","2017","2016","2015","2014","2013"]: 
    try:
        pit_df = pd.read_excel("../resources/2007-2020-PIT-Estimates-by-CoC.xlsx",sheet,skipfooter=3)

        # Rename pit_df columns
        pit_df.rename(columns = {'CoC Number' : 'Coc',f'Overall Homeless, {sheet}' : "Total",f'Unsheltered Homeless, {sheet}' : "Unsheltered"},inplace=True)

        # Create New Columns
        pit_df["Sheltered"] = pit_df["Total"]-pit_df["Unsheltered"]
        pit_df = pit_df[['Coc','Total','Unsheltered','Sheltered']]
        pit_df["Year"] = sheet
        pit_df["CocYear"] = pit_df["Coc"] +" "+ pit_df["Year"]

        # Create HomelessData table
        pit_df.to_sql("HomelessCounts",con=conn,if_exists='append',index=False)
        print(f"Gathered Data from {sheet}")
    except Exception as E : 
        print(f"Couldnt get data from {sheet}")
print("Finished")

Gathered Data from 2020
Gathered Data from 2019
Gathered Data from 2018
Gathered Data from 2017
Gathered Data from 2016
Gathered Data from 2015
Gathered Data from 2014
Gathered Data from 2013
Finished


In [5]:
# Merging the Homeless counts and Shelter avalibility tables
c.execute("""
CREATE TABLE CocData AS
SELECT * FROM ShelterAvalibility
JOIN HomelessCounts
ON ShelterAvalibility.CocYear = HomelessCounts.CocYear
""")
c.execute("ALTER TABLE CocData DROP 'CocYear:1';")
c.execute("ALTER TABLE CocData DROP 'Coc:1';")
c.execute("ALTER TABLE CocData DROP 'Year:1';")

In [6]:
# Create The Funding Database and Dataframe
for sheet in ["2020","2019","2018","2017","2016","2015","2014","2013"]:
    try:
        awards_df = pd.read_excel("../resources/2019-Awards-by-CoC-Component-Project.xlsx",sheet)
        awards_df.rename(columns = {"CoC Number" : "CoC"},inplace=True)
        awards_df = awards_df[["CoC","Component","Awarded Amount"]]
        awards_df["Year"] = sheet
        awards_df.to_sql("FundingData",con=conn,if_exists='append',index=False)
        print(f"Gathered Data from {sheet}")
    except Exception as E : 
        print(f"Couldnt gather data from {sheet}")
        
print("Finished")

Couldnt gather data from 2020


C:\Users\smshe\anaconda3\envs\mlenv\lib\site-packages\pandas\core\generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


Gathered Data from 2019
Gathered Data from 2018
Couldnt gather data from 2017
Couldnt gather data from 2016
Couldnt gather data from 2015
Couldnt gather data from 2014
Couldnt gather data from 2013
Finished


In [7]:
# Create the Coc Meta
df = pd.read_excel("../resources/2007-2020-PIT-Estimates-by-CoC.xlsx",skipfooter=3)[['CoC Number',"CoC Category"]]
df.to_sql("CocMeta",con=conn,index=False)

In [8]:
# Create CountyMeta
county_meta_df = pd.read_csv("../Resources\county_level_data\county_coc_source.csv")
lat_lng_df = pd.read_csv("../Resources\county_level_data\wiki_county_lat_long.csv",skiprows = [1])[["FIPS","Latitude","Longitude"]]
df = pd.merge(county_meta_df,lat_lng_df, on="FIPS")
df.rename(columns = {"number" : "Coc"},inplace=True)
df.to_sql("CountyMeta",con=conn,index=False)

In [9]:
# Create County Data Table
county_data_df = pd.read_csv("../Resources\county_level_data\county_data.csv")
clean_df = pd.DataFrame()
for column in ["2019","2018","2017","2016","2015","2014","2013"]: 
    df = county_data_df[["FIPS",f"{column}_population",f"Unemployment_rate_{column}"]]
    df.rename(columns = {f"{column}_population" : "Population",f"Unemployment_rate_{column}" : "Unemploment"},inplace=True)
    df["year"] = column
    df.to_sql("CountyData",if_exists = "append",con=conn,index=False)
    print(f"Gathered Data from {column}")
print("Finished")

Gathered Data from 2019
Gathered Data from 2018
Gathered Data from 2017
Gathered Data from 2016
Gathered Data from 2015
Gathered Data from 2014
Gathered Data from 2013
Finished


C:\Users\smshe\anaconda3\envs\mlenv\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\smshe\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [10]:
pd.read_sql("SELECT * FROM CountyData",con=conn)

,FIPS,Population,Unemploment,year
0,1001,"55,869",2.7,2019
1,1003,"223,234",2.8,2019
2,1005,"24,686",3.8,2019
3,1007,"22,394",3.1,2019
4,1009,"57,826",2.7,2019
...,...,...,...,...
21975,56037,"45,157",4.4,2013
21976,56039,"22,315",5.7,2013
21977,56041,"20,951",5.3,2013
21978,56043,"8,413",5.0,2013
